In [6]:
%pip install rasterio numpy scikit-image scikit-learn


     ---------------------------------------- 24.5/24.5 MB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 9.2/9.2 MB 6.4 MB/s eta 0:00:00
     --------------------------------------- 44.1/44.1 MB 12.3 MB/s eta 0:00:00
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
     ------------------------------------- 313.2/313.2 kB 18.9 MB/s eta 0:00:00
     ---------------------------------------- 222.9/222.9 kB ? eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\mucky\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\networkx\\utils\\tests\\test_misc.py'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scikit_image-0.22.0-cp311-cp311-win_amd64.whl (24.5 MB)
  Using cached scikit_learn-1.3.1-cp311-cp311-win_amd64.whl (9.2 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached imageio-2.31.5-py3-none-any.whl (313 kB)
  Using cached lazy_loader-0.3-py3-none-any.whl (9.1 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import rasterio
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.model_selection import train_test_split

# Step 1: Define a function to read and extract the required bands from the directory

def extract_bands_from_directory(directory):
    bands = {}
    for file in os.listdir(directory):
        if file.endswith(".tif"):
            band_number = file.split('.')[-2][-3:]
            with rasterio.open(os.path.join(directory, file)) as src:
                bands[band_number] = src.read(1)
    return bands

def normalize_band(band):
    scaler = MinMaxScaler()
    band_normalized = scaler.fit_transform(band)
    return band_normalized

def save_tile(output_path, bands, profile):
    with rasterio.open(output_path, 'w', **profile) as dest:
        for i, band in enumerate(bands, 1):
            dest.write(band, i)

def preprocess_hls_data(directory):
    bands = extract_bands_from_directory(directory)

    necessary_bands = ['B02', 'B03', 'B04', 'B08', 'B11', 'B12']
    
    if not all(band in bands for band in necessary_bands):
        print(f"Skipped {directory}, missing necessary bands.")
        return None

    bands_normalized = [normalize_band(bands[band]) for band in necessary_bands]

    with rasterio.open(os.path.join(directory, next(file for file in os.listdir(directory) if file.endswith(".tif")))) as src:
        profile = src.profile
        profile.update(count=len(bands_normalized))

    return bands_normalized, profile

def split_into_tiles(image, size=512):
    height, width = image.shape
    tiles = []
    for i in range(0, height, size):
        for j in range(0, width, size):
            tile = image[i:i+size, j:j+size]
            if tile.shape[0] == size and tile.shape[1] == size:
                tiles.append(tile)
    return tiles

input_dir = 'C:\\Users\\mucky\\Hackz\\nasadatasethls\\hls-bulk-download\\outdir'
output_dirs = {'train': 'train_data', 'val': 'val_data', 'test': 'test_data'}

for output_type, output_dir in output_dirs.items():
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

data_dirs = [subdir for subdir, _, files in os.walk(input_dir) if any(file.endswith('.tif') for file in files)]
train_dirs, test_dirs = train_test_split(data_dirs, test_size=0.2, random_state=42)
train_dirs, val_dirs = train_test_split(train_dirs, test_size=0.25, random_state=42)

for data_type, dirs in [('train', train_dirs), ('val', val_dirs), ('test', test_dirs)]:
    for directory in dirs:
        result = preprocess_hls_data(directory)
        if result:
            bands_normalized, profile = result
            for i, band in enumerate(bands_normalized):
                tiles = split_into_tiles(band)
                for j, tile in enumerate(tiles):
                    output_filename = f"subsetted_512x512_{os.path.basename(directory)}_B{i+1}_tile{j}.tif"
                    output_path = os.path.join(output_dirs[data_type], output_filename)
                    save_tile(output_path, [tile], profile)
                    print(f"Processed {directory} to {data_type}, tile {j+1}")

Processed C:\Users\mucky\Hackz\nasadatasethls\hls-bulk-download\outdir\S30\2021\37\Q\G\C\HLS.S30.T37QGC.2021186T074619.v2.0 to train, tile 1
Processed C:\Users\mucky\Hackz\nasadatasethls\hls-bulk-download\outdir\S30\2021\37\Q\G\C\HLS.S30.T37QGC.2021186T074619.v2.0 to train, tile 2
Processed C:\Users\mucky\Hackz\nasadatasethls\hls-bulk-download\outdir\S30\2021\37\Q\G\C\HLS.S30.T37QGC.2021186T074619.v2.0 to train, tile 3
Processed C:\Users\mucky\Hackz\nasadatasethls\hls-bulk-download\outdir\S30\2021\37\Q\G\C\HLS.S30.T37QGC.2021186T074619.v2.0 to train, tile 4
Processed C:\Users\mucky\Hackz\nasadatasethls\hls-bulk-download\outdir\S30\2021\37\Q\G\C\HLS.S30.T37QGC.2021186T074619.v2.0 to train, tile 5
Processed C:\Users\mucky\Hackz\nasadatasethls\hls-bulk-download\outdir\S30\2021\37\Q\G\C\HLS.S30.T37QGC.2021186T074619.v2.0 to train, tile 6
Processed C:\Users\mucky\Hackz\nasadatasethls\hls-bulk-download\outdir\S30\2021\37\Q\G\C\HLS.S30.T37QGC.2021186T074619.v2.0 to train, tile 7
Processed C:\